### Khai báo thư viện

In [188]:
!pip install emoji
!pip install pyvi

In [189]:
import pandas as pd
import emoji
import unicodedata
import regex as re
from pyvi import ViTokenizer, ViPosTagger

### Kết nối với gg drive

In [190]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Đọc file data.csv

In [191]:
df = pd.read_csv('/content/drive/MyDrive/data.csv')

In [192]:
df.head(10)

,URL,Content
0,https://www.facebook.com/share/p/14DqtuJaps/,"Mọi người ơiii, hãy vào link này để sở hữu bộ ..."
1,https://www.facebook.com/share/p/14DqtuJaps/,Nhạc vẫn đỉnh như ngày nào
2,https://www.facebook.com/share/p/14DqtuJaps/,Chúc mừng Sơn Tùng M-TP và MB Bank
3,https://www.facebook.com/share/p/14DqtuJaps/,Ok zai iu của mẹ
4,https://www.facebook.com/share/p/14DqtuJaps/,mê nhạc mê anhhhh
5,https://www.facebook.com/share/p/14DqtuJaps/,Xịn quáaaa
6,https://www.facebook.com/share/p/14DqtuJaps/,Tuyệt vời a iu
7,https://www.facebook.com/share/p/14DqtuJaps/,Mê a quá phải ls ạ
8,https://www.facebook.com/share/p/14DqtuJaps/,Ước gì anh tùng được làm chủ tịch nước
9,https://www.facebook.com/share/p/14DqtuJaps/,Bài hát hay tràn đầy năng lượng nha!!!


In [193]:
df.shape

(9255, 2)

In [194]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9255 entries, 0 to 9254
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   URL      9255 non-null   object
 1   Content  9008 non-null   object
dtypes: object(2)
memory usage: 144.7+ KB


### Làm sạch data

In [195]:
bang_nguyen_am= [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]

bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']
nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

Đổi thành chữ thường hết

In [196]:
def chuyen_chu_thuong(text):
	return text.lower()

Chuẩn hóa unicode. Có 2 loại unicode : unicode tổ hơp và unicode dựng sẵn, điêu này dẫn tới việc 2 từ giống nhau sẽ bị coi là khác nhau. Chuẩn hóa tất cả về 1 loại là unicode dựng sẵn

In [197]:
def chuan_hoa_unicode(text):
    """
    Chuẩn hóa Unicode an toàn, xử lý các giá trị NaN và float

    Parameters:
    text: Giá trị cần chuẩn hóa

    Returns:
    Giá trị đã được chuẩn hóa
    """
    # Kiểm tra nếu là NaN hoặc float
    if pd.isna(text):
        return text

    # Chuyển đổi sang chuỗi
    text = str(text)

    # Chuẩn hóa Unicode
    text = unicodedata.normalize('NFC', text)

    return text

Có 2 kiểu gõ dấu ở Tiếng Việt, ví dụ như là : òa hoặc oà (ta gọi lần lượt là chuẩn 1 và 2). Mặc dù kiểu gõ chữ sau ít phổ biến hơn tuy nhiên vẫn cần phải chuẩn hóa tránh việc một số văn bản vẫn sử dụng kiểu gõ dấu thứ 2.

In [198]:
"""
	Hàm này xử lý chuẩn hóa từng từ một, sau khi chuẩn hóa từng từ thì ta sẽ đi chuân hóa từng câu sau
"""
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)

In [199]:
def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

In [200]:
def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = str(sentence)
    words = sentence.split()
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\\p{P}*)([p{L}.]*\\p{L}+)(\\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

Tách từ tiếng việt, từ tiếng việt không giống như tiếng anh, tách từ tiếng anh ta chỉ cần tách bằng khoảng trắng
Tuy nhiên từ tiếng Việt có cả từ đơn lẫn từ ghép nên tách từ tiêng Việt sẽ phúc tạp hơn
Project sử dung thu viện pyvi (xem mã nguồn tại : https://github.com/trungtv/pyvi) để phục vụ bài toán con tách từ Tiếng Việt

In [201]:
def tach_tu_tieng_viet(text):
	text = ViTokenizer.tokenize(text)
	return text

Xóa đi các dấu cách thừa, các từ không cần thiết cho việc phân loại vẳn bản

In [202]:
def chuan_hoa_cau(text):
	text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_.,]',' ',text)
	text = re.sub(r'\s+', ' ', text).strip()
	return text

Loại bỏ link

In [203]:
def clean_link(text):
  if pd.isna(text):
      return text
  # Loại bỏ các link http, https, www
  text = re.sub(r'http[s]?://\S+|www\.\S+', '', str(text))
  # Loại bỏ các link không có giao thức
  text = re.sub(r'\b\w+\.\w+/\S+', '', text)
  return text.strip()

Loại bỏ số

In [204]:
def clean_numbers(text):
  if pd.isna(text):
      return text

  # Loại bỏ tất cả các chữ số
  text = re.sub(r'\d+', '', str(text))

  # Loại bỏ khoảng trắng thừa
  text = ' '.join(text.split())

  return text.strip()

Thêm khoảng cách cho dấu "." và dấu ","

In [205]:
def xu_ly_dau_cau(text):
        if pd.isna(text):
            return text

        # Chuyển sang chuỗi
        text = str(text)

        # Thêm khoảng cách hai bên dấu chấm và dấu phẩy
        # Đảm bảo không thêm khoảng cách giữa số thập phân
        text = re.sub(r'([.,])(?!\d)', r' \1 ', text)

        # Loại bỏ dấu " và dấu '
        text = text.replace('"', '')
        text = text.replace("'", '')

        # Loại bỏ khoảng trắng thừa
        text = re.sub(r'\s+', ' ', text).strip()

        return text.strip()

In [206]:
test = '"haha'
print(test)
print(xu_ly_dau_cau(test))

"haha
haha


Các xử lý cần trong bài toán này

In [207]:
def tien_xu_li(text):
  text = chuan_hoa_unicode(text)
  text = clean_link(text)
  text = clean_numbers(text)
  text = xu_ly_dau_cau(text)
  text = chuan_hoa_dau_cau_tieng_viet(text)
  # text = tach_tu_tieng_viet(text)
  text = chuyen_chu_thuong(text)
  text = chuan_hoa_cau(text)

  return text

In [212]:
df_clean = df.copy()
df_clean['Content'] = df_clean['Content'].apply(tien_xu_li)

In [213]:
df_clean.shape

(9255, 2)

Loại bỏ content trùng lặp



In [214]:
df_clean = df_clean.drop_duplicates(subset=['Content'])

In [215]:
df_clean.shape

(8045, 2)

In [217]:
df_clean.head(10)

,URL,Content
0,https://www.facebook.com/share/p/14DqtuJaps/,"mọi người ơiii , hãy vào link này để sở hữu bộ..."
1,https://www.facebook.com/share/p/14DqtuJaps/,nhạc vẫn đỉnh như ngày nào
2,https://www.facebook.com/share/p/14DqtuJaps/,chúc mừng sơn tùng m tp và mb bank
3,https://www.facebook.com/share/p/14DqtuJaps/,ok zai iu của mẹ
4,https://www.facebook.com/share/p/14DqtuJaps/,mê nhạc mê anhhhh
5,https://www.facebook.com/share/p/14DqtuJaps/,xịn quáaaa
6,https://www.facebook.com/share/p/14DqtuJaps/,tuyệt vời a iu
7,https://www.facebook.com/share/p/14DqtuJaps/,mê a quá phải ls ạ
8,https://www.facebook.com/share/p/14DqtuJaps/,ước gì anh tùng được làm chủ tịch nước
9,https://www.facebook.com/share/p/14DqtuJaps/,bài hát hay tràn đầy năng lượng nha


### Lưu lại data vừa làm sạch

In [216]:
df_clean.to_csv('data_cleaned.csv', index=False)